In [1]:
import cvxpy as cp
import numpy as np

# DGP Approach 1 - Beta as variable

In [193]:
k_cats = cp.Variable(6, pos=True) # (1/sec)
K_ms = cp.Variable(6, pos=True) # K_M (mol/L)
BETA   = cp.Variable(1, pos=True) # unitless
# k_cats_diff_slack = cp.Variable(1, pos=True) # (1/sec)
# K_m_diff_slack = cp.Variable(1, pos=True) # (mol/L)
S = cp.Variable(6, pos=True) # (mol/L)

P_L = 0.00014218052336201800 # (mol/L)
k_cat_exp = 0.071 # (1/sec)
K_ms_exp = 3.7E-6 # (mol/L)
#PD03938, G6890-MONOMER, G6737-MONOMER, RPOD-MONOMER, PD02936, RED-THIOREDOXIN2-MONOMER
S_protein_concs = np.array([4.46510054478746E-07, 0.00027087422911447700, 0.0006261898295372690, 0.0006151283915983840, 8.64621184397196E-06, 0.0006831411710823680]) # (mol/L)
S_std_dev = np.array([6.48605511482799E-07, 0.00017348313510493000, 0.00014728328233160000, 0.0002649777812708530, 2.04927424163342E-05, 0.00013085185087382300]) # (mol/L)
t_half = np.array([79.1510021, 214.018094, 288.738238, 503.910765, 468.6027322, 367.223275])*60 # (sec)

# construct constraints
constraints = [BETA >= cp.sum(cp.multiply(cp.inv_pos(K_ms),S)) + 1,
               S <= S_protein_concs + S_std_dev]
for idx, (concs, dev) in enumerate(zip(S_protein_concs, S_std_dev)):
    if concs - dev > 0:
        constraints += [S[idx] >= concs - dev]
    else:
        constraints += [S[idx] >= 0.01 * concs]
# if S_protein_concs[0] - S_std_dev[0] > 0:
#     constraints += [S >= S_protein_concs - S_std_dev]
# else:
#     constraints += [S >= 0.01 * S_protein_concs]
# constraints += [S >= 0.5 * S_protein_concs]
constraints += [cp.multiply(P_L*k_cats, cp.inv_pos(K_ms * BETA)) >= np.log(2)/t_half]

# construct objective
rel_diff_k_cats = cp.maximum(k_cats / k_cat_exp, k_cat_exp / k_cats) 
rel_diff_K_ms = cp.maximum(K_ms / K_ms_exp, K_ms_exp / K_ms)

obj_k_cats = cp.sum(rel_diff_k_cats) # minimize the sum of relative difference k_cats
obj_K_ms = cp.sum(rel_diff_K_ms) # minimize the sum of relative difference K_ms
obj_deg_rate = cp.sum(cp.multiply(P_L*k_cats, cp.inv_pos(K_ms * BETA))) # minimize to achieve ..
objectives = [obj_k_cats, obj_K_ms, obj_deg_rate]

# construct problem
problem = cp.Problem(cp.Minimize(obj_k_cats + obj_K_ms + obj_deg_rate + BETA), constraints)
problem.solve(gp=True, solver=cp.CLARABEL)

86.10093306423687

In [194]:
print('Simulated value of beta:', BETA.value)
print('Simulated value of k_cats:', k_cats.value) 
print('Simulated value of K_ms:', K_ms.value) # show 2 decimal places
print('Simulated value of S:', S.value)
print('Simulated difference between obj_deg_rate and ln(2)/half_life:', obj_deg_rate.value - np.sum(np.log(2)/t_half))

Simulated value of beta: [39.5443234]
Simulated value of k_cats: [0.071 0.071 0.071 0.071 0.071 0.071]
Simulated value of K_ms: [3.70000004e-06 1.89666166e-05 4.20152305e-05 3.59292970e-05
 3.70000005e-06 4.51182510e-05]
Simulated value of S: [4.46527853e-09 9.73910950e-05 4.78906550e-04 3.50150613e-04
 8.64622635e-08 5.52289323e-04]
Simulated difference between obj_deg_rate and ln(2)/half_life: 0.16996764807725184


In [195]:
S_protein_concs

array([4.46510054e-07, 2.70874229e-04, 6.26189830e-04, 6.15128392e-04,
       8.64621184e-06, 6.83141171e-04])

In [196]:
obj_deg_rate.value

0.17028662917139

In [197]:
np.sum(np.log(2)/t_half)

0.0003189810941381753

In [95]:
BETA.value, np.sum(np.multiply(1/(K_ms.value),S.value)) + 1

(array([39.5443234]), 39.54432292639752)

In [30]:
rel_diff_k_cats.value

# DGP Appraoch 2 - Beta as parameter

In [203]:
k_cats = cp.Variable(6, pos=True) # (1/sec)
K_ms = cp.Variable(6, pos=True) # K_M (mol/L)
S = cp.Variable(6, pos=True) # (mol/L)

P_L = 0.00014218052336201800 # (mol/L)
k_cat_exp = 0.071 # (1/sec)
K_ms_exp = 3.7E-6 # (mol/L)
#PD03938, G6890-MONOMER, G6737-MONOMER, RPOD-MONOMER, PD02936, RED-THIOREDOXIN2-MONOMER
S_protein_concs = np.array([4.46510054478746E-07, 0.00027087422911447700, 0.0006261898295372690, 0.0006151283915983840, 8.64621184397196E-06, 0.0006831411710823680]) # (mol/L)
S_std_dev = np.array([6.48605511482799E-07, 0.00017348313510493000, 0.00014728328233160000, 0.0002649777812708530, 2.04927424163342E-05, 0.00013085185087382300]) # (mol/L)
t_half = np.array([79.1510021, 214.018094, 288.738238, 503.910765, 468.6027322, 367.223275])*60 # (sec)

# construct constraints
# 1. Define beta as a parameter
beta = cp.sum(cp.multiply(cp.inv_pos(K_ms),S)) + 1

# 2. Define range of feasible substrate concentrations
constraints = [S <= S_protein_concs + S_std_dev]
for idx, (concs, dev) in enumerate(zip(S_protein_concs, S_std_dev)):
    if concs - dev > 0:
        constraints += [S[idx] >= concs - dev]
    else:
        constraints += [S[idx] >= 0.01 * concs]

# 3. Define relationship between k_cats, K_ms, and half life that we try to minimize the difference of 
constraints += [cp.multiply(P_L*k_cats, cp.inv_pos(K_ms * beta)) >= np.log(2)/t_half]

constraints += [k_cats >= k_cat_exp] # add constraint on k_cats
# construct objective
# 1. Define relative error between estimation and experimental values
rel_diff_k_cats = cp.maximum(k_cats / k_cat_exp, k_cat_exp / k_cats) 
rel_diff_K_ms = cp.maximum(K_ms / K_ms_exp, K_ms_exp / K_ms)
obj_k_cats = cp.sum(rel_diff_k_cats) # minimize the sum of relative difference k_cats
obj_K_ms = cp.sum(rel_diff_K_ms) # minimize the sum of relative difference K_ms

# 2. Define objective to minimize the degradation rate
k_degs = [cp.quad_over_lin(P_L*k_cats[i], cp.inv_pos(K_ms[i] * beta)) for i in range(6)]
obj_deg_rate = cp.sum(k_degs) # minimize to achieve degradation rate
objectives = [obj_k_cats, obj_K_ms, obj_deg_rate]

# construct problem
problem = cp.Problem(cp.Minimize(obj_k_cats + obj_K_ms + obj_deg_rate), constraints)
problem.solve(gp=True, solver=cp.CLARABEL)

12.000000014182937

In [204]:
print('Simulated value of beta:', beta.value)
print('Simulated value of k_cats:', k_cats.value) 
print('Simulated value of K_ms:', K_ms.value) # show 2 decimal places
print('Simulated value of S:', S.value)
print('Simulated difference between obj_deg_rate and ln(2)/half_life:', obj_deg_rate.value - np.sum(np.log(2)/t_half))

Simulated value of beta: 497.61419370801417
Simulated value of k_cats: [0.071 0.071 0.071 0.071 0.071 0.071]
Simulated value of K_ms: [3.7e-06 3.7e-06 3.7e-06 3.7e-06 3.7e-06 3.7e-06]
Simulated value of S: [7.36131357e-07 1.83295263e-04 5.59012611e-04 4.49440584e-04
 1.62156143e-05 6.28772313e-04]
Simulated difference between obj_deg_rate and ln(2)/half_life: -0.00031898109301242353


In [205]:
# Solve for problem without obj_k_cats and obj_K_ms
problem = cp.Problem(cp.Minimize(obj_deg_rate), constraints)
problem.solve(gp=True, solver=cp.CLARABEL)

5.629708783161636e-13

In [207]:
print('Simulated value of beta:', beta.value)
print('Simulated value of k_cats:', k_cats.value) 
print('Simulated value of K_ms:', K_ms.value) # show 2 decimal places
print('Simulated value of S:', S.value)
print('Simulated difference between obj_deg_rate and ln(2)/half_life:', obj_deg_rate.value - np.sum(np.log(2)/t_half))

Simulated value of beta: 250801922.3311549
Simulated value of k_cats: [0.07100005 0.071      0.071      0.071      0.07100001 0.071     ]
Simulated value of K_ms: [1.98004046e-14 2.92458290e-12 6.48556638e-12 5.54538677e-12
 8.71391375e-14 6.96466635e-12]
Simulated value of S: [4.46510645e-09 9.73910954e-05 4.78906551e-04 3.50150613e-04
 8.64621455e-08 5.52289324e-04]
Simulated difference between obj_deg_rate and ln(2)/half_life: -0.0003189810935752044


In [141]:
obj_deg_rate.value

1.3111515163148117e-17

In [175]:
S[0].value/K_ms[0].value

58350.37733353076

In [190]:
t = k_cats <= 1
t.is_dgp()

True

In [ ]:
temp = k_cats-k_cat_exp
temp

In [ ]:
temp.is_dqcp()

In [ ]:
temp = k_cats-k_cat_exp <= k_cats_diff_slack
temp

In [ ]:
temp.is_dcp()

In [ ]:

# consatruct problem
# constraint = [
#     S_protein_concs - S_std_dev >= cp.quad_over_lin(cp.sqrt(k_cats), K_ms), # (mol/L)
#     # P_L * k_cats/(k_dogs*beta) <= S_protein_concs + S_std_dev,
# ]
constraints = []
constraints +=  [cp.inv_pos(1/K_ms) + 1 <= beta]
for i, (concs, std) in enumerate(zip(S_protein_concs, S_std_dev)):
    upper = concs + std
    if concs - std > 0:
        lower = concs - std
    else:
        lower = 0.01*concs
    constraints += [cp.inv_pos(K_ms[i]*beta) * k_cats[i] * P_L <= upper]
    constraints += [cp.inv_pos(K_ms[i]*beta) * k_cats[i] * P_L >= lower]


assert all(constraint.is_dgp() for constraint in constraints)
# for constraint in constraints:
#     print(constraint.is_dgp())
#     print(constraint)
#     print(constraint.is_dcp())

objective = cp.Minimize(beta+ cp.norm1(k_cats-k_cat_est) + cp.norm1(K_ms-K_ms_est)) # minimize the sum of k_cats
problem = cp.Problem(objective, constraints)
assert problem.is_dgp()

In [ ]:
temp = cp.norm1(k_cats-k_cat_est)
temp.is_log_log_convex()

In [ ]:
cp.sum(cp.inv_pos(1/K_ms)) + 1

In [ ]:
temp = cp.sum(cp.inv_pos(1/K_ms)) + 1 >= beta
temp

In [ ]:
temp.is_dgp()

In [ ]:
f = cp.sum(cp.inv_pos(1/K_ms)) + 1
f.is_dgp()
f.is_log_log_convex()

In [ ]:
beta.is_log_log_affine()

In [ ]:
f <= beta

In [ ]:
f >= beta

In [ ]:
beta <= f